In [5]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/tcc')
'''

import json
from tqdm import tqdm

from modules.preprocessing import *


In [14]:
## Teste
'''
phrase = (  "German Chancellor Angela Merkel and US President Barack Obama "
            "converse in the Oval Office inside the White House in Washington, D.C. "
            "health care not work here @moisesoliv")
prepare_text = Tokenizer()
wordList = prepare_text.cleanText(phrase)
wordList
'''

'\nphrase = (  "German Chancellor Angela Merkel and US President Barack Obama "\n            "converse in the Oval Office inside the White House in Washington, D.C. "\n            "health care not work here @moisesoliv")\nprepare_text = Tokenizer()\nwordList = prepare_text.cleanText(phrase)\nwordList\n'

In [3]:
prepare_text = Tokenizer()
with open('dataset/unified-dataset.jsonl', 'r') as json_file:
    json_list = list(json_file)

datasets = {'text': list(),
            'tokens': list(),
            'source': list(),
            'anger': list(),
            'disgust': list(),
            'fear': list(),
            'joy': list(),
            'love': list(),
            'noemo': list(),
            'sadness': list(),
            'surprise': list()}

for json_str in tqdm(json_list):
    result = json.loads(json_str)
    datasets['text'].append(result['text'])
    datasets['tokens'].append(prepare_text.cleanText(result['text']))
    datasets['source'].append(result['source'])
    datasets['anger'].append(result['emotions']['anger'])
    datasets['disgust'].append(result['emotions']['disgust'])
    datasets['fear'].append(result['emotions']['fear'])
    datasets['joy'].append(result['emotions']['joy'])
    datasets['love'].append(result['emotions']['love'])
    datasets['noemo'].append(result['emotions']['noemo'])
    datasets['sadness'].append(result['emotions']['sadness'])
    datasets['surprise'].append(result['emotions']['surprise'])


100%|██████████| 221439/221439 [32:00<00:00, 115.30it/s]


In [ ]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
save_object(datasets, 'dataset/organized_dataset.pkl')

## abaixo daqui nao é tão importante


In [ ]:
import pickle
import pandas as pd
from tqdm import tqdm

datasets = pickle.load(open("dataset/organized_dataset.pkl", "rb"))

df = pd.DataFrame(datasets, columns = ['text', 'tokens', 'source', 'label', 'anger','disgust','fear', 'joy', 'love', 'noemo', 'sadness', 'surprise']) 
df.drop_duplicates(['source'])

,text,tokens,source,label,anger,disgust,fear,joy,love,noemo,sadness,surprise
0,"Remember what she said in my last letter? ""","[rememb, what, she, said, in, my, last, letter]",emobank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10062,@PressSec sounds so defensive &amp; combative...,"[twitter_usernam, sound, so, defens, , amp, co...",grounded_emotions,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.00,NaN
12647,@tiffanylue i know i was listenin to bad habi...,"[twitter_usernam, i, know, , i, wa, listenin, ...",crowdflower,NaN,0.00,0.00,0.0,0.0,0.0,1.0,0.00,0.0
52387,"I'm a #Republican, but if I have to hear my mo...","[i, 'm, a, republican, but, if, i, have, to, h...",electoraltweets,NaN,0.00,1.00,0.0,0.0,NaN,0.0,0.00,0.0
56443,Thinks that @melbahughes had a great 50th birt...,"[think, that, twitter_usernam, had, a, great, ...",tec,NaN,0.00,0.00,0.0,0.0,NaN,NaN,0.00,1.0
77494,Mortar assault leaves at least 18 dead,"[mortar, assault, leav, atleast18, dead]",affectivetext,NaN,0.22,0.02,0.6,0.0,NaN,NaN,0.64,0.0
78744,He who exalts himself shall be humbled; a...,"[he, who, exalt, himself, shall, , be, humbl, ...",ssec,NaN,1.00,1.00,0.0,1.0,NaN,NaN,0.00,0.0
83612,This did the trick : the boys now have a more ...,"[thi, did, the, trick, the, boy, now, have, a,...",emotion-cause,NaN,0.00,0.00,0.0,1.0,NaN,NaN,0.00,0.0
86026,The kitchen stinks .,"[the, kitchen, stink]",dailydialog,NaN,0.00,1.00,0.0,0.0,NaN,0.0,0.00,0.0
189005,@ZubairSabirPTI pls dont insult the word 'Molna',"[twitter_usernam, , pl, do, nt, insult, the, w...",emoint,NaN,1.00,NaN,0.0,0.0,NaN,NaN,0.00,NaN


In [ ]:
import gensim
size_word2vec = 300
vectors_name = "ft_unified"+str(size_word2vec)+".pkl"
try:
    w2v = pickle.load(open(vectors_name, "rb"))
except:
    w2v = gensim.models.Word2Vec(df['tokens'].values,size=size_word2vec, workers=8, compute_loss=True, )
    save_object(w2v, vectors_name)



SyntaxError: ignored

In [ ]:
training_loss = w2v.get_latest_training_loss()
print(training_loss)


In [ ]:
print(w2v.wv.most_similar(positive=['woman', 'king'], negative=['man']))
print(w2v.wv.most_similar('gr8'))
print(w2v.wv.most_similar('newyork'))
print(w2v.wv.most_similar('twitter_usernam'))

In [ ]:
crowdflower = df.loc[df['source'].isin(['dailydialog', 'crowdflower', 'electoraltweets'])]
crowdflower

In [ ]:
labels = ['anger'] #, 'disgust', 'fear', 'joy', 'sadness', 'surprise']
crowdflower[labels].sum()

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

def getIndex(t):
    try:
        return w2v.wv.vocab[t].index
    except:
        return 0

sequences = [[getIndex(t) for t in comment]
             for comment in crowdflower['tokens']]

maxlen = MAX_SEQUENCE_LENGTH = 35
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre", truncating="post")

y = crowdflower[labels].values
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

# y = np.reshape(y, [len(labels), len(y)])
# y_train.shape


x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.20, random_state=42)
# y_train = np.reshape(y_train, [len(labels), len(y_train)])
# y_train.shape

# y_test = np.reshape(y_test, [len(labels), len(y_test)])
# y_test.shape
MAX_NB_WORDS = len(w2v.wv.vocab)
nb_words = min(MAX_NB_WORDS, len(w2v.wv.vocab))


In [ ]:
import tensorflow as tf
MAX_NB_WORDS = len(w2v.wv.vocab)
nb_words = min(MAX_NB_WORDS, len(w2v.wv.vocab))

def model_gru(embedding_matrix, embed_size):
    outp = list()
    inp = tf.keras.layers.Input(shape=(maxlen,))
    x = tf.keras.layers.Embedding(nb_words, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    for l in labels:
        layers['x'+ l]= tf.keras.layers.Bidirectional(tf.keras.layers.GRU(35, return_sequences=True))(x)
        layers['avg_pool'+ l] = tf.keras.layers.GlobalAveragePooling1D()(layers['x'+ l])
        layers['max_pool'+ l] = tf.keras.layers.GlobalMaxPooling1D()(layers['x'+ l])
        layers['conc'+ l] = tf.keras.layers.concatenate([layers['avg_pool'+ l], layers['max_pool'+ l]])
        layers['conc'+ l] = tf.keras.layers.Dense(70, activation="relu")(layers['conc'+ l])

        layers['conc'+ l] = tf.keras.layers.Dropout(0.5)(layers['conc'+ l])
        outp.append(tf.keras.layers.Dense(1, activation="sigmoid")(layers['conc'+ l]))
    model = tf.keras.models.Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = model_gru(w2v.wv.vectors, 300)

tf.keras.utils.plot_model(model,show_shapes=True, expand_nested=False)


In [ ]:
import random
import numpy as np

def balanced_batch_generator(x, y, batch_size, categorical=True):
    """A generator for creating balanced batched.

    This generator loops over its data indefinitely and yields balanced,
    shuffled batches.

    Args:
    x (numpy.ndarray): Samples (inputs). Must have the same length as `y`.
    y (numpy.ndarray): Labels (targets). Must be a binary class matrix (i.e.,
        shape `(num_samples, num_classes)`). You can use `keras.utils.to_categorical`
        to convert a class vector to a binary class matrix.
    batch_size (int): Batch size.
    categorical (bool, optional): If true, generates binary class matrices
        (i.e., shape `(num_samples, num_classes)`) for batch labels (targets).
        Otherwise, generates class vectors (i.e., shape `(num_samples, )`).
        Defaults to `True`.
    Returns a generator yielding batches as tuples `(inputs, targets)` that can
        be directly used with Keras.
    """
    if x.shape[0] != y.shape[0]:
        raise ValueError('Args `x` and `y` must have the same length.')
    if len(y.shape) != 2:
        raise ValueError(
            'Arg `y` must have a shape of (num_samples, num_classes). ' +
            'You can use `keras.utils.to_categorical` to convert a class vector ' +
            'to a binary class matrix.'
        )
    if batch_size < 1:
        raise ValueError('Arg `batch_size` must be a positive integer.')
    num_samples = y.shape[0]
    num_classes = y.shape[1]
    batch_x_shape = (batch_size, *x.shape[1:])
    batch_y_shape = (batch_size, num_classes) if categorical else (batch_size, )
    indexes = [0 for _ in range(num_classes)]
    samples = [[] for _ in range(num_classes)]
    for i in range(num_samples):
        samples[np.argmax(y[i])].append(x[i])
    while True:
        batch_x = np.ndarray(shape=batch_x_shape, dtype=x.dtype)
        batch_y = np.zeros(shape=batch_y_shape, dtype=y.dtype)
        for i in range(batch_size):
            random_class = random.randrange(num_classes)
            current_index = indexes[random_class]
            indexes[random_class] = (current_index + 1) % len(samples[random_class])
            if current_index == 0:
                random.shuffle(samples[random_class])
            batch_x[i] = samples[random_class][current_index]
            if categorical:
                batch_y[i][random_class] = 1
            else:
                batch_y[i] = random_class
        yield (batch_x, batch_y)

In [ ]:
from imblearn.under_sampling import NearMiss

# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
x_test, y_test = undersample.fit_resample(x_test, y_test)


# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
x_train, y_train = undersample.fit_resample(x_train, y_train)
'''
hist = model.fit_generator(generator=balanced_batch_generator(x_train, y_train,20),
                            steps_per_epoch=1,
                            validation_data = (x_test, y_test),
                            epochs=20, verbose=1)
'''

hist = model.fit(x_train, y_train,
                    validation_split=0.1,
                    epochs=10,
                    batch_size=30,
                    verbose=1)


In [ ]:
model.summary()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])


In [ ]:
save_object(model, 'convlsdtmw2v50.pkl')

In [ ]:
model = pickle.load(open("convlsdtmw2v50.pkl", "rb"))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# rus = RandomUnderSampler(return_indices=True)
# x_rus, y_rus, id_rus = rus.fit_sample(x_test, y_test)

# print('Removed indexes:', id_rus)
predictions = model.predict(x_test)
y_pred = (predictions > 0.5)
# confusion_matrix(y_test, y_pred)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
matrix = matrix/matrix.astype(np.float).sum(axis=1)
sns.heatmap(matrix)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")


plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], 
            loc='upper left')
plt.show()

plt.plot(hist.history[MAX_NB_WORDS = len(w2v.wv.vocab)
'loss'])
plt.plot(hist.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
 # More metrics
# https://scikit-learn.org/stable/modules/model_evaluation.html
# https://nbviewer.jupyter.org/github/autonomio/talos/blob/master/examples/Hyperparameter%20Optimization%20with%20Keras%20for%20the%20Iris%20Prediction.ipynb#seven

labels[1]